<a href="https://colab.research.google.com/github/SwatiNeha/neural-networks-applied/blob/main/Search_for_best_hyperparameters%20for%20Spherical%20Projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code for HyperParameter Tuning using Grid Search**

In [ ]:
import torch as tp
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import plotly.graph_objects as go

seed = 42
np.random.seed(seed)
tp.manual_seed(seed)

def generate_norm_samples(data_size):
    X = tp.normal(mean=0.0, std=1.0, size=(data_size, 3))
    return X

def generate_output_sphere(data):
    y = data / tp.norm(data, dim=1, keepdim=True)
    return y

data_size = 10000
X = generate_norm_samples(data_size)
y = generate_output_sphere(X)

X_train_init, X_test, y_train_init, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_init, y_train_init, test_size=0.25, random_state=42)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(3, 20)
        self.layer2 = nn.Linear(20, 20)
        self.layer3 = nn.Linear(20, 3)

    def forward(self, x):
        x = tp.relu(self.layer1(x))
        x = tp.relu(self.layer2(x))
        x = self.layer3(x)
        return x

NN_model = NeuralNetwork()

# PyTorch Regressor with grid search parameters
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, epochs=10, batch_size=32, learning_rate=0.01):
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model = NeuralNetwork()
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.train_losses = []
        self.valid_losses = []

    def fit(self, X, y):
        X = tp.tensor(X, dtype=tp.float32)
        y = tp.tensor(y, dtype=tp.float32)

        for epoch in range(self.epochs):
            self.model.train()
            choices = tp.randperm(X.size()[0])
            epoch_loss = 0.0

            for i in range(0, X.size()[0], self.batch_size):
                index = choices[i:i+self.batch_size]
                x1, y1 = X[index], y[index]

                self.optimizer.zero_grad()
                outputs = self.model(x1)
                loss = self.criterion(outputs, y1)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

            train_loss = epoch_loss / (X.size()[0] / self.batch_size)
            self.train_losses.append(train_loss)

            # Validation loss
            self.model.eval()
            with tp.no_grad():
                valid_X = tp.tensor(X_val.clone().detach().numpy(), dtype=tp.float32)
                valid_y = tp.tensor(y_val.clone().detach().numpy(), dtype=tp.float32)
                valid_outputs = self.model(valid_X)
                valid_loss = self.criterion(valid_outputs, valid_y)
                self.valid_losses.append(valid_loss.item())

        return self

    def predict(self, X):
        X = tp.tensor(X, dtype=tp.float32)
        self.model.eval()
        with tp.no_grad():
            outputs = self.model(X)
        return outputs.numpy()

param_grid = {
    'epochs': [25, 50, 100, 200],
    'batch_size': [16, 25, 32, 64],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Grid Search
grid_search = GridSearchCV(estimator=PyTorchRegressor(), param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)

X_train_all = np.concatenate((X_train, X_val))
y_train_all = np.concatenate((y_train, y_val))

# Performing grid search
grid_search.fit(X_train_all, y_train_all)

# Best Parameters
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

best_model = PyTorchRegressor(**best_params)
best_model.fit(X_train_all, y_train_all)

# Evaluating on test data
y_predicted = best_model.predict(X_test.numpy())
testing_loss = np.mean((y_predicted - y_test.numpy())**2)
print(f'Test Loss: {testing_loss}')


Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] END ........batch_size=16, epochs=25, learning_rate=0.1; total time=  19.4s
[CV] END ........batch_size=16, epochs=25, learning_rate=0.1; total time=  12.9s
[CV] END ........batch_size=16, epochs=25, learning_rate=0.1; total time=  12.4s
[CV] END .......batch_size=16, epochs=25, learning_rate=0.01; total time=  12.6s
[CV] END .......batch_size=16, epochs=25, learning_rate=0.01; total time=  12.5s
[CV] END .......batch_size=16, epochs=25, learning_rate=0.01; total time=  13.2s
[CV] END ......batch_size=16, epochs=25, learning_rate=0.001; total time=  13.2s
[CV] END ......batch_size=16, epochs=25, learning_rate=0.001; total time=  12.5s
[CV] END ......batch_size=16, epochs=25, learning_rate=0.001; total time=  12.8s
[CV] END ........batch_size=16, epochs=50, learning_rate=0.1; total time=  25.6s
[CV] END ........batch_size=16, epochs=50, learning_rate=0.1; total time=  25.4s
[CV] END ........batch_size=16, epochs=50, lear